In [1]:
import arcpy
from arcpy import env
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???)

In [2]:
# # store paths to parcels
# year = 2019
# p_web = r"E:\Data\Parcels_2019\Parcels_Weber_LIR.gdb\Parcels_Weber_LIR"
# p_sl = r"E:\Data\Parcels_2019\Parcels_SaltLake_LIR.gdb\Parcels_SaltLake_LIR"
# p_ut = r"E:\Data\Parcels_2019\Parcels_Utah_LIR.gdb\Parcels_Utah_LIR"
# p_dav = r"E:\Data\Parcels_2019\Parcels_Davis_LIR.gdb\Parcels_Davis_LIR"

In [3]:
# store paths to parcels
year = 2015
p_web = r"E:\Data\Parcels_Jan_2016\Parcels_Weber_LIR.gdb\Weber_Parcels_2016"
p_sl = r"E:\Data\Parcels_Jan_2016\Parcels_SaltLake_LIR.gdb\SaltLake_Parcels_2016"
p_ut = r"E:\Data\Parcels_Jan_2016\Parcels_Utah_LIR.gdb\Utah_Parcels_2016"
p_dav = r"E:\Data\Parcels_Jan_2016\Parcels_Davis_LIR.gdb\Davis_Parcels_2016"

In [4]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')

# create output gdb
outputs = ['.\\Outputs', "percentiles.gdb"]
gdb = os.path.join(outputs[0], outputs[1])
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

In [5]:
remm_parcels_lyr = arcpy.MakeFeatureLayer_management('E:\REMM\Boundaries\Center01102019.gdb\BaseGP01102019_utm12', 
                                                     'remm_parcels_lyr')

# query for only single family and parcels that have a building
query = (""" type1 = 't' And type2 = 'f' And type3 = 'f' And type4 = 'f' And 
            type5 = 'f' And type6 = 'f' And type7 = 'f' And type8 = 'f' And basebldg = 1""")
arcpy.SelectLayerByAttribute_management(remm_parcels_lyr, 'NEW_SELECTION', query)

# create layers for each set of county parcels
p_web_lyr = arcpy.MakeFeatureLayer_management(p_web,'p_web_lyr')
p_sl_lyr = arcpy.MakeFeatureLayer_management(p_sl,'p_sl_lyr')
p_ut_lyr = arcpy.MakeFeatureLayer_management(p_ut,'p_ut_lyr')
p_dav_lyr = arcpy.MakeFeatureLayer_management(p_dav,'p_dav_lyr')

# get sf parcels
arcpy.SelectLayerByLocation_management(in_layer=p_web_lyr, 
                                       overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=remm_parcels_lyr,
                                       selection_type='NEW_SELECTION')

# get sf parcels
arcpy.SelectLayerByLocation_management(in_layer=p_sl_lyr, 
                                       overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=remm_parcels_lyr,
                                       selection_type='NEW_SELECTION')

# get sf parcels
arcpy.SelectLayerByLocation_management(in_layer=p_ut_lyr, 
                                       overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=remm_parcels_lyr,
                                       selection_type='NEW_SELECTION')

# get sf parcels
arcpy.SelectLayerByLocation_management(in_layer=p_dav_lyr, 
                                       overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=remm_parcels_lyr,
                                       selection_type='NEW_SELECTION')

# get sf lir parcels
p_web_sf = arcpy.FeatureClassToFeatureClass_conversion(p_web_lyr, gdb, '_01_weber_sf')
p_sl_sf = arcpy.FeatureClassToFeatureClass_conversion(p_sl_lyr, gdb, '_01_saltlake_sf')
p_ut_sf = arcpy.FeatureClassToFeatureClass_conversion(p_ut_lyr, gdb, '_01_utah_sf')
p_dav_sf = arcpy.FeatureClassToFeatureClass_conversion(p_dav_lyr, gdb, '_01_davis_sf')

# merge em
sf_parcels = arcpy.Merge_management([p_web_sf, p_sl_sf, p_ut_sf, p_dav_sf], os.path.join(gdb, '_02_sf_parcels'))
sf_parcels_lyr = arcpy.MakeFeatureLayer_management(sf_parcels,'sf_parcels_lyr')

In [ ]:
sf_class_polygons = r'E:\Projects\Create_Residential_Capacity_Classes\Outputs\classes.gdb\single_family_residential_polygons'
sf_class_polygons_lyr = arcpy.MakeFeatureLayer_management(sf_class_polygons,'sf_class_polygons_lyr')

codes = ['a1', 'a2', 'a3', 'a4', 'a5', 
           'b1', 'b2', 'b3', 'b4', 'b5', 
           'c1', 'c2', 'c3', 'c4', 'c5', 
           'd1', 'd2', 'd3', 'd4', 'd5', 
           'e1', 'e2', 'e3', 'e4', 'e5']

table = pd.DataFrame(codes, columns=['code'])
table['second'] = None
table['fifth'] = None
table['tenth'] = None
table['median'] = None
table['min'] = None
table['max'] = None
table['count'] = None


# calc percentiles and descriptive statistics for total value per acre, across parcels, in each class
for code in codes:
    
    # query for current class
    query = (""" code = '{}' """.format(code))
    arcpy.SelectLayerByAttribute_management(sf_class_polygons_lyr, 'NEW_SELECTION', query)
#     arcpy.FeatureClassToFeatureClass_conversion(sf_class_polygons_lyr, gdb, '_03_class_{}'.format(code))

    # get sf parcels
    arcpy.SelectLayerByLocation_management(in_layer=sf_parcels_lyr, 
                                           overlap_type="HAVE_THEIR_CENTER_IN",
                                           select_features=sf_class_polygons_lyr,
                                           selection_type='NEW_SELECTION')
    
    # get parcels that are large than .05 acres
    query = (""" PARCEL_ACRES >= 0.05 """)
    arcpy.SelectLayerByAttribute_management(sf_parcels_lyr, 'SUBSET_SELECTION', query)
#     arcpy.FeatureClassToFeatureClass_conversion(sf_parcels_lyr, gdb, '_04_sf_parcels_{}'.format(code))
    
    # count features
    count = arcpy.GetCount_management(sf_parcels_lyr)[0]
    
    # read straight from layer into dbf
    code_parcels_sdf = pd.DataFrame.spatial.from_featureclass(sf_parcels_lyr)
    code_parcels_sdf = code_parcels_sdf[['OBJECTID','BUILT_YR','COUNTY_NAME',
                                         'TOTAL_MKT_VALUE','LAND_MKT_VALUE','PARCEL_ACRES']].copy()
    
    # Calulate total market value per acre and get rows greater than zero
    code_parcels_sdf['TMV_ACRE'] = code_parcels_sdf['TOTAL_MKT_VALUE'] / code_parcels_sdf['PARCEL_ACRES']
    code_parcels_sdf = code_parcels_sdf[code_parcels_sdf['TOTAL_MKT_VALUE'] > 0]
    
    # percentiles
    second_per = code_parcels_sdf['TMV_ACRE'].quantile(.025)
    fifth_per = code_parcels_sdf['TMV_ACRE'].quantile(.05)
    tenth_per = code_parcels_sdf['TMV_ACRE'].quantile(.1)
    median = code_parcels_sdf['TMV_ACRE'].quantile(.5)
    minimum = code_parcels_sdf['TMV_ACRE'].min()
    maximum = code_parcels_sdf['TMV_ACRE'].max()
    
    # assign the value
    table.loc[(table['code'] == code), 'second'] = second_per
    table.loc[(table['code'] == code), 'fifth'] = fifth_per
    table.loc[(table['code'] == code), 'tenth'] = tenth_per
    table.loc[(table['code'] == code), 'median'] = median
    table.loc[(table['code'] == code), 'min'] = minimum
    table.loc[(table['code'] == code), 'max'] = maximum
    table.loc[(table['code'] == code), 'count'] = count

In [6]:
# export to csv
table.to_csv('Outputs\\percentiles_{}.csv'.format(year))